In [64]:
#  뉴스 기사의 댓글 모으기 - 관광 
# 테스트 기사 URL : https://news.v.daum.net/v/20190730050115159

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math
import numpy  
import pandas as pd  
import random
import os
import re

#Step 2. 사용자에게 검색어 키워드를 입력 받고 저장할 폴더와 파일명을 설정합니다.
print("=" *80)
print(" 연습문제 17-4: 다음 카카오 뉴스 기사의 댓글 정보  수집하기")
print("=" *80)
print("\n")

query_txt = '뉴스기사댓글'
query_url = input('1.댓글을 크롤링할 뉴스의 URL을 입력하세요: ')
cnt = int(input('2.크롤링 할 건수는 몇건입니까?(10건단위로 입력요망): '))
page_cnt = math.ceil(cnt / 10)

f_dir = input("3.파일을 저장할 폴더명만 쓰세요(예:c:\\temp\\):")

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.

s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get(query_url)
time.sleep(5)

#Step 4. 현재 총 리뷰 건수를 확인하여 사용자의 요청건수와 비교 후 동기화합니다
import re
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

result= soup.find('div', class_='cmt_count_box').find('em','num_count')
result2 = result.get_text()

print("=" *80)
result3 = result2.replace(",","")
result4 = re.search("\d+",result3)
search_cnt = int(result4.group())

if cnt > search_cnt :
    cnt = search_cnt

print("전체 검색 결과 건수 :",search_cnt,"건")
print("실제 최종 출력 건수",cnt)
print("실체 출력될 최종 페이지수" , page_cnt)

 연습문제 17-4: 다음 카카오 뉴스 기사의 댓글 정보  수집하기


1.댓글을 크롤링할 뉴스의 URL을 입력하세요: https://news.v.daum.net/v/20190730050115159
2.크롤링 할 건수는 몇건입니까?(10건단위로 입력요망): 100
3.파일을 저장할 폴더명만 쓰세요(예:c:\temp\):c:\data\
전체 검색 결과 건수 : 7420 건
실제 최종 출력 건수 100
실체 출력될 최종 페이지수 10


In [65]:
# Step 5. 사용자가 요청한 건수가 많을 경우 리뷰 더보기 버튼을 클릭합니다
# 기사 아래에 최초 3건의 댓글이 있고 그 아래 더보기 버튼을 누르면 10건씩 추가로 댓글이 보입니다.

#사용자에게 입력 받은 건수를 10으로 나누어 page_cnt 값에 넣은 만큼  더보기 버튼을 누르면서 자동 스크롤 다운 합니다

def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(2)
            
   
i = 1
while (i <= page_cnt):
    scroll_down(driver) 
    i += 1
    driver.find_element_by_xpath("""//*[@id="alex-area"]/div/div/div/div[3]/div[1]/a""").click()

#Step 6. 20건 출력되어 있는 현재 페이지 리뷰와 점수 등 내용 수집
writer_id2=[]
review2=[]
write_date2=[]
gogam=[]
gogam_0=[]
gogam_1=[]

f = open(ff_name, 'a',encoding='UTF-8')
    
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
    
count = 0

#전체 댓글 정보를 가져옵니다 
reple_result = soup.find('ul', class_='list_comment')
slist = reple_result.find_all('li')

print(len(slist))

for li in slist:
        count += 1
        print("\n")
        print("%s 번째 댓글 수집 중 ==================" %count)
        f.write("\n" + "총 %s 건 중 %s 번째 리뷰 데이터를 수집합니다==============" %(cnt,count) )
        
        #작성자ID추출
        writer_id = li.find('span', class_='info_author').find('a').get_text()
        print("1.작성자ID:", writer_id)
        f.write("\n")
        f.write("1.작성자ID:"+writer_id + "\n")
        writer_id2.append(writer_id)
        
        # 리뷰 내용 추출
        try :
            review = li.find('div', class_='cmt_info').find('p','desc_txt font_size_17').get_text().replace("\n","").strip()
        except AttributeError :
            review='작성자에 의해 삭제된 댓글입니다'
            print("2.리뷰 :",review)
        else :
            print("2.리뷰:",review)
        
        f.write("2.리뷰:" + review + "\n")
        review2.append(review)
 

        # 작성일자 추출
        write_date = li.find('span', class_='info_author').find('span','txt_date').get_text()
        print('3.작성일자:',write_date)
        f.write("3.작성일자:" + write_date + "\n")
        write_date2.append(write_date)
              

        #공감 / 비공감 추출
        gogam = li.find('span', class_='comment_recomm').find_all('span','num_txt')
        
        try :
            g_gogam = gogam[0].text
            print('4.공감:',g_gogam)
        except  :
            g_gogam = '0'
            print('4.공감 :',g_gogam)
        f.write("4.공감:" + g_gogam + "\n")
        gogam_0.append(g_gogam)
        
        try :
            b_gogam = gogam[1].text
            print('5.비공감:',b_gogam) 
        except :
            b_gogam = '0'
            print('5.비공감 :',b_gogam)
        f.write("5.비공감:" + b_gogam + "\n")
        gogam_1.append(b_gogam)
              
        time.sleep(0.2)        
    
        if count == cnt :
             break
                   
print("%s 건  완료========================================================" %count)
time.sleep(random.randrange(3,8))  # 3-8 초 사이에 랜덤으로 시간 선택                                       

       
# 학습목표 3. 수집된 데이터를 표 형태로 저장하기
#Step 7. xls 형태와 csv 형태로 저장하기

news_reple = pd.DataFrame()
news_reple['작성자ID']=pd.Series(writer_id2)
news_reple['리뷰내용']=pd.Series(review2)
news_reple['작성일자']=pd.Series(write_date2)
news_reple['공감횟수']=pd.Series(gogam_0)
news_reple['비공감횟수']=pd.Series(gogam_1)

# csv 형태로 저장하기
news_reple.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
news_reple.to_excel(fx_name ,index=True)

# Step 8. 요약 정보 출력하기

e_time = time.time( )
t_time = e_time - s_time

print("\n") 
print("=" *80)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )

93


1 번째 댓글 수집 중 ==================
1.작성자ID: 베스트
2.리뷰: 제주도는 도지사를바꿔야 한다
3.작성일자: 7시간전
4.공감: 18036
5.비공감: 549


2 번째 댓글 수집 중 ==================
1.작성자ID: 이담
2.리뷰: 해외여행이 늘어난 원인중 하나가 제주도 동해안쪽 관광지의 바기지다.
3.작성일자: 6시간전
4.공감: 12561
5.비공감: 46


3 번째 댓글 수집 중 ==================
1.작성자ID: 다소니
2.리뷰: 제주도 정신차려야한다.
3.작성일자: 6시간전
4.공감: 5606
5.비공감: 47


4 번째 댓글 수집 중 ==================
1.작성자ID: 나그네
2.리뷰: 중앙 기레기야 국내여행이 비싸고 맘에 안들면동남아나 베트남으로 가자는 거다이시기에  꼭 이따위 글을 써야하냐일본은 가지 말자고!
3.작성일자: 6시간전
4.공감: 3466
5.비공감: 365


5 번째 댓글 수집 중 ==================
1.작성자ID: 상선약수
2.리뷰: 누가 어플리케이션 좀 만들어 주세요 ~‘ 바가지 및 안 바가지 ^^ ‘어느 골짜기 평상, 닭백숙 얼마,어느 마을 펜션 1박에 얼마,어느 식당 밥값 얼마 ~민박집 방값 얼마 ~ ...아울러 어느집, 어느 동네 가격 대비 품질이우수한 곳도 함께 올려주는 앱을 만들어 줬으면 합니다.
3.작성일자: 6시간전
4.공감: 1753
5.비공감: 10


6 번째 댓글 수집 중 ==================
1.작성자ID: KCS
2.리뷰: 그래서  일본  관광 가야?쫑 암   기레기  .친일  매국 일보
3.작성일자: 6시간전
4.공감: 1085
5.비공감: 136


7 번째 댓글 수집 중 ==================
1.작성자ID: 뱅이
2.리뷰: 누가 제주가랬냐? 일본말고 다른 곳 가라고 했지!왜구 신문, 중앙아!
3.작성일자: 6시간전
4.공감: 1014
5.비공감: 86


8 번째 댓글 수집 



53 번째 댓글 수집 중 ==================
1.작성자ID: 독수리 공방
2.리뷰: 조중동 눔들이 이젠 일존을 안가니까 자국 제주도를 흠잡으면서 일본여행이 더좋다고 말하는고아니냐
3.작성일자: 6시간전
4.공감: 5
5.비공감: 1


54 번째 댓글 수집 중 ==================
1.작성자ID: 스카
2.리뷰: 제주도는 사실 외국이지. 그 나라 사람들도 그렇게 생각하더구만...
3.작성일자: 6시간전
4.공감: 5
5.비공감: 1


55 번째 댓글 수집 중 ==================
1.작성자ID: 차밍맨
2.리뷰: 그런데 이 중앙일보 기자쉐끼는 내용이 뭐야 ?일본 관광상품 설명회 하는거야 ?이런 ♩♪♪~♬♩ ~똥통에 처박아 죽일넘 같으니.암튼 국가의 미래를 위해서는 하루 빨리 조,중,동 을 없애야 한다.
3.작성일자: 3시간전
4.공감: 4
5.비공감: 0


56 번째 댓글 수집 중 ==================
1.작성자ID: jcpwin
2.리뷰: 난 제주간다.애들한테 방사능 먹이긴 싫고 동남아 덥고 습하고관광지에서 식사만 안하면 가성비 괜찮음랜트카 싸고 숙소 싸고 비싸다 하기전에 검색을 하시요
3.작성일자: 4시간전
4.공감: 4
5.비공감: 0


57 번째 댓글 수집 중 ==================
1.작성자ID: 돌쇠
2.리뷰: 제주가 좋아서 내려온지 4년차 입니다. 넘 맞있고 저렴한곳이 지천인데요 우리 가족은 점심 7-8천원 정도면 아주 훌륭합니다. 숨어있는 맞집이 많은데요 어떻게 취재되었는지 궁금합니다. 비싼 집들만 골라서?제주도 홈페이지 관광을 클릭하면 거기에도 많고요 그곳에 소개된 곳 외에도 싸고 맛좋은 집들이 찿으면 너무 많습니다.
3.작성일자: 4시간전
4.공감: 4
5.비공감: 0


58 번째 댓글 수집 중 ==================
1.작성자ID: 꼬막이
2.리뷰: 바가지....20년전 야그입니다....모두 자한당과 왜놈과 기레기 프레임에 놀아난 형태입니다..흉기차도 마찬가